In [325]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import re
import shutil
# import string
# import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# import re
import collections
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [259]:
train_df=pd.read_csv('../data/processed_train.csv')

In [260]:
test_df=pd.read_csv('../data/processed_test.csv')

In [261]:
from list_to_corpus import *
from helper import *

In [262]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 3263 non-null   int64 
 1   text               3263 non-null   object
 2   modified_keyword   3263 non-null   object
 3   modified_location  3263 non-null   object
 4   url_count          3263 non-null   int64 
 5   urls               3263 non-null   object
 6   emoji_count        3263 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 178.6+ KB


In [268]:
%%time
train_df['text']=train_df.text.fillna('uns_text')
test_df['text']=test_df.text.fillna('uns_text')

CPU times: user 3.34 ms, sys: 264 µs, total: 3.6 ms
Wall time: 3.49 ms


In [269]:
train_df.head()

,id,text,target,modified_keyword,modified_location,url_count,urls,emoji_count
0,1,Our Deeds are the Reason of this #earthquake M...,1,uns_keyword,uns_location,0,uns_url,0
1,4,Forest fire near La Ronge Sask. Canada,1,uns_keyword,uns_location,0,uns_url,0
2,5,All residents asked to 'shelter in place' are ...,1,uns_keyword,uns_location,0,uns_url,0
3,6,"13,000 people receive #wildfires evacuation or...",1,uns_keyword,uns_location,0,uns_url,0
4,7,Just got sent this photo from Ruby #Alaska as ...,1,uns_keyword,uns_location,0,uns_url,0


In [270]:
%%time
train_corpus=list_corpus(train_df['text'])
test_corpus=list_corpus(test_df['text'])

CPU times: user 19.7 s, sys: 208 ms, total: 19.9 s
Wall time: 21 s


In [327]:
len(train_corpus), len(test_corpus)

(7613, 3263)

In [271]:
word = [doc.split() for doc in test_corpus]
lst = []
for item in word:
    for i in item:
        lst.append(i)

In [272]:
len(collections.Counter(lst))

13166

In [273]:
train_target = train_df.pop('target')

In [275]:
# train_df

In [329]:
%%time
vectorizer = TfidfVectorizer(max_features=10000)
tf_train_X = vectorizer.fit_transform(train_corpus)
tf_test_X = vectorizer.transform(test_corpus)

CPU times: user 394 ms, sys: 114 ms, total: 508 ms
Wall time: 683 ms


In [330]:
tf_test_X.shape, tf_train_X.shape

((3263, 10000), (7613, 10000))

## Embedding Model (only text features)

In [280]:
embedding_dim = 128
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 128)         128128    
_________________________________________________________________
dropout_18 (Dropout)         (None, None, 128)         0         
_________________________________________________________________
global_average_pooling1d_4 ( (None, 128)               0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 128,257
Trainable params: 128,257
Non-trainable params: 0
_________________________________________________________________


In [404]:
METRICS = [
            tf.metrics.BinaryAccuracy(name='ACCURACY'),
            tf.metrics.Precision(name='PRECISION'),
            tf.metrics.Recall(name='RECALL'),
            tf.metrics.AUC(name='AUC'),
            tf.metrics.TruePositives(name='TP'),
            tf.metrics.TrueNegatives(name='TN'),
            tf.metrics.FalsePositives(name='FP'),
            tf.metrics.FalseNegatives(name='FN')]

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=METRICS)

In [405]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(train_X, train_target.values, verbose=1, batch_size = 200, epochs = 2, validation_split=0.2,
                   shuffle=True, callbacks=[tensorboard_callback])

Epoch 1/2


InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential_4/dense_9/BiasAdd:0) = ] [[-0.257828951][-0.278070807][-0.270402223]...] [y (Cast_8/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]] [Op:__inference_train_function_60708]

Function call stack:
train_function


In [382]:
# history_dict = history.history
# history_dict.keys()

In [383]:
# fig, ax = plt.subplots(1,2,figsize=(15,4))

# acc = history_dict['binary_accuracy']
# val_acc = history_dict['val_binary_accuracy']
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(acc) + 1)

# # "bo" is for "blue dot"
# ax[0].plot(epochs, loss, 'bo', label='Training loss')
# # b is for "solid blue line"
# ax[0].plot(epochs, val_loss, 'ro', label='Validation loss')
# ax[0].set_title('Training and validation loss (only text features)')
# ax[0].set_xlabel('Epochs')
# ax[0].set_ylabel('Loss')
# ax[0].legend()

# # "bo" is for "blue dot"
# ax[1].plot(epochs, acc, 'bo', label='Training accuracy')
# # b is for "solid blue line"
# ax[1].plot(epochs, val_acc, 'ro', label='Validation accuracy')
# ax[1].set_title('Training and validation accuracy (only text features)')
# ax[1].set_xlabel('Epochs')
# ax[1].set_ylabel('Accuracy')
# ax[1].legend()

# plt.tight_layout()


In [335]:
np.unique(model.predict(test_X))

array([-0.33723179, -0.33720806, -0.33720803], dtype=float32)

## Embedding model (all features)

In [205]:
test_df

,id,text,modified_keyword,modified_location,url_count,urls,emoji_count
0,1,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url,0
1,4,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url,0
2,5,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url,0
3,6,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url,0
4,7,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url,0
...,...,...,...,...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,uns_keyword,uns_location,0,uns_url,0
7609,10870,@aria_ahrary @TheTawniest The out of control w...,uns_keyword,uns_location,0,uns_url,0
7610,10871,M1.94 [0104 UTC]?5km S of Volcano Hawaii.,uns_keyword,uns_location,1,http//t.co/zDtoyd8EbJ,1
7611,10872,Police investigating after an e-bike collided ...,uns_keyword,uns_location,0,uns_url,0


In [406]:
vectorizer = CountVectorizer(max_features=100, binary=True)
train_dummy_keyword = vectorizer.fit_transform(list_corpus(train_df['modified_keyword'])).toarray()
test_dummy_keyword = vectorizer.transform(list_corpus(test_df['modified_keyword'])).toarray()

In [407]:
vectorizer = CountVectorizer(max_features=100, binary=True)
train_dummy_location = vectorizer.fit_transform(list_corpus(train_df['modified_location'])).toarray()
test_dummy_location = vectorizer.transform(list_corpus(test_df['modified_location'])).toarray()

In [408]:
vectorizer = CountVectorizer(max_features=100, binary=True)
train_dummy_urls = vectorizer.fit_transform(list_corpus(train_df['urls'])).toarray()
test_dummy_urls = vectorizer.transform(list_corpus(test_df['urls'])).toarray()

In [350]:
train_dummy_keyword.shape, test_dummy_keyword.shape

((7613, 100), (3263, 100))

In [409]:
train_X_augmented = np.hstack([train_X, train_dummy_location, train_dummy_keyword, train_dummy_urls])

In [410]:
test_X_augmented = np.hstack([test_X, test_dummy_location, test_dummy_keyword, test_dummy_urls])

In [412]:
train_X_augmented.shape, test_X_augmented.shape

((7613, 10300), (3263, 10300))

In [413]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
history_augmented = model.fit(train_X_augmented, train_target.values, verbose=1, batch_size = 200, epochs = 1, validation_split=0.2,
                   shuffle=True, callbacks=[tensorboard_callback])

InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential_4/dense_9/BiasAdd:0) = ] [[-0.312711746][-0.300694913][-0.262787968]...] [y (Cast_8/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]] [Op:__inference_train_function_61946]

Function call stack:
train_function


In [324]:
train_X_augmented.shape, test_X_augmented.shape

((7613, 15289), (3263, 13536))

In [355]:
np.unique(model.predict(test_X))

array([-0.31817454, -0.31815702], dtype=float32)

# Naive Bayes

In [414]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [415]:
clf = GaussianNB()

In [416]:
clf.fit(train_X_augmented, train_target.values)

GaussianNB()

In [417]:
prediction_=clf.predict(test_X_augmented)

In [418]:
np.mean(clf.predict(test_X_augmented))

0.34201654918786395

In [419]:
train_X_augmented.shape

(7613, 10300)

In [420]:
clf = BernoulliNB()

In [421]:
clf.fit(train_X_augmented, train_target.values)

BernoulliNB()

In [422]:
prediction_=clf.predict(test_X_augmented)

In [424]:
pd.Series(prediction_).to_csv('../rg_bernoulliNB_submission.csv')